## YΣ19 Artificial Intelligence II
# Homework 2

### Iglezou Myrto - 111520170038

# Model 1

In [1]:
import pandas as pd 
from pandas import DataFrame, read_csv

In [2]:
# import gdown
# url = 'https://drive.google.com/file/d/1dTIWNpjlrnTQBIQtaGOh0jCRYZiAQO79/view?usp=sharing'
# path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
# output = "twitterData.csv"
# gdown.download(path, output, quiet=False)
# twitterData = pd.read_csv("/content/twitterData.csv")
# df = pd.DataFrame(data=twitterData)
# df.head(5)

path = r"C:\Users\myrto\Documents\GitHub\AI2-project2\SentimentTweets.csv"
file = pd.read_csv(path,index_col=False,nrows=50000)
# file = pd.read_csv(path,index_col=False)
df = pd.DataFrame(data=file)
df.head(5)

,Unnamed: 0,target,id,date,flag,user,text
0,680949,0,2249621587,Fri Jun 19 22:41:08 PDT 2009,NO_QUERY,sukumarpant,#brokenpromises...
1,406741,0,2059003515,Sat Jun 06 16:03:21 PDT 2009,NO_QUERY,MTMSparrow,David Carradine so sad. Thai's law not sure i...
2,1337108,4,2017466467,Wed Jun 03 08:26:14 PDT 2009,NO_QUERY,itsmemcee,A @ 415 B @ 425. Tell your bro i say congrats!
3,1560887,4,2186457254,Mon Jun 15 18:52:04 PDT 2009,NO_QUERY,jdfreivald,@littlefluffycat Indeed.
4,1466295,4,2064458395,Sun Jun 07 06:19:20 PDT 2009,NO_QUERY,CrazyHan,Completed Race 4 Life in 58mins with girlies f...


In [3]:
df['target'].replace({4: 1}, inplace = True)
X = df['text']
y = df['target']

## Data pre-processing

### Removing the scpecial characters

In [4]:
import re

def RemoveSpecialCharacters(x):
    x = re.sub(r'[^\w]+', " ", x)
    return x

In [5]:
X = X.apply(lambda x: RemoveSpecialCharacters(x) )

### Lemmatize

In [6]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

X = X.apply(lambda x: WordNetLemmatizer().lemmatize(x))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\myrto\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Removing the upercase

In [7]:
X = X.apply(lambda x: x.lower())

### Create a list o all words

In [8]:
words = []
for row in X:
    sentence = row.split()
    for word in sentence:
        words.append(word)
        
words.append("<pad>")

### Create the vocabulary

In [9]:
from collections import Counter

counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
word_2_id = {w:i for i,w in enumerate(vocab,1)}

### Encode the words

In [10]:
def convert_tokens_to_inds(sentence, word_2_id):
    return [word_2_id.get(t) for t in sentence]

In [11]:
X = X.apply(lambda x: x.split())

In [12]:
X = X.apply(lambda x: convert_tokens_to_inds(x, word_2_id))

In [13]:
X

0                                                  [16916]
1        [873, 4516, 18, 112, 2751, 12, 1510, 26, 192, ...
2        [4, 16919, 315, 16920, 294, 46, 1037, 1, 202, ...
3                                            [16921, 1135]
4        [3860, 1267, 154, 177, 10, 16922, 23, 5502, 54...
                               ...                        
49995               [3904, 214, 190, 1478, 163, 1198, 106]
49996    [58980, 87, 77, 1, 20, 26, 478, 18, 22, 2140, ...
49997    [1, 129, 107, 651, 15, 154, 210, 14, 117, 225,...
49998          [13382, 8, 7, 2659, 881, 3404, 19, 8, 2764]
49999                      [58981, 166, 8, 11, 139, 4, 98]
Name: text, Length: 50000, dtype: object

### Check for empty rows

In [14]:
rows = Counter([len(x) for x in X])
print("Zero-length sentences: {}".format(rows[0]))

Zero-length sentences: 0


### Padding for windows.

Find the maximum sentence lenght and make every sentence that long, by adding 0 on the right and left.

In [15]:
def pad_sentence_for_window(sentence, max_row, pad_token=0):
    padding = max_row - len(sentence)
    
    left = (int(padding/2) if padding%2 ==0 else int(padding/2 + 1) )
    right = int(padding/2)
    return [pad_token]*left + sentence + [pad_token]*right

In [16]:
max_row = max(rows)

X = X.apply(lambda x: pad_sentence_for_window(x,max_row))

### Transform data to a 2D array

In [17]:
import numpy as np
X = np.vstack(X.values)

In [18]:
y = y.values.tolist()
y = np.array(y)

### Split data

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4, stratify = y)

### Batching sentences together with a DataLoader

In [20]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Tensor datasets
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

# dataloaders
batch_size = 32

# SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

## Classifier - Model1

In [21]:
import torch.nn as nn

class Model1(nn.Module):
    def __init__(self, vocab_size, input_dim, hidden_dim, output_size, n_layers):
        
        #constructor
        super(Model1,self).__init__()
        
        #embedding layer
#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #dense layer
        self.fc = nn.Linear(input_dim, output_size)
        
        #activation function
        self.sig = nn.Sigmoid()

    def forward(self, x):
#         x = self.embedding(x)
        x = self.fc(x)
        x = self.sig(x)
#         x = self.output(x)
#         x = self.softmax(x)
        return x


### Instantiate the network

In [22]:
vocab_size = len(word_2_id) + 1 # +1 for zero padding + our word tokens
output_size = 1
hidden_dim = 200 
input_dim = X_train.shape[1]
n_layers = 2

net = Model1(vocab_size, input_dim, hidden_dim, output_size, n_layers)

print(net)

Model1(
  (fc): Linear(in_features=73, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [23]:
import torch.optim as optim

lr=0.001
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

### Score function 

In [24]:
def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

## Training

In [25]:

num_epochs = 50

list_of_losses = []
list_of_acc = []

for epoch in range(num_epochs):
    running_loss = 0.0
    running_acc = 0.0
    net.train()
    for batch_index, batch in tqdm(enumerate(train_loader)):
        x_train, y_train = batch[0], batch[1]
        x_train = x_train.float()
        y_train = y_train.float()
        
        # step 1. zero the gradients
        optimizer.zero_grad()

        # step 2. compute the output
        y_pred = net(x_train)

        # step 3. compute the loss
        loss = loss_function(y_pred.squeeze(), y_train)
        loss_t = loss.item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)

        # step 4. use loss to produce gradients
        loss.backward()

        # step 5. use optimizer to take gradient step
        optimizer.step()
        # -----------------------------------------
        # compute the accuracy
        acc_t = compute_accuracy(y_pred, y_train)
        running_acc += (acc_t - running_acc) / (batch_index + 1)
   
    list_of_losses.append(running_loss)
    list_of_acc.append(running_acc)


1250it [00:02, 503.62it/s]
1250it [00:02, 537.17it/s]
1250it [00:02, 532.61it/s]
1250it [00:02, 540.99it/s]
1250it [00:02, 533.21it/s]
1250it [00:02, 530.51it/s]
1250it [00:01, 692.29it/s]
1250it [00:01, 631.15it/s]
1250it [00:01, 755.81it/s]
1250it [00:01, 697.72it/s]
1250it [00:01, 710.97it/s]
1250it [00:01, 796.60it/s]
1250it [00:01, 746.82it/s]
1250it [00:01, 721.40it/s]
1250it [00:01, 745.68it/s]
1250it [00:01, 757.54it/s]
1250it [00:01, 708.48it/s]
1250it [00:01, 738.42it/s]
1250it [00:01, 798.99it/s]
1250it [00:01, 677.08it/s]
1250it [00:01, 702.79it/s]
1250it [00:01, 712.07it/s]
1250it [00:01, 639.75it/s]
1250it [00:01, 678.90it/s]
1250it [00:01, 801.77it/s]
1250it [00:01, 773.84it/s]
1250it [00:01, 739.16it/s]
1250it [00:01, 651.12it/s]
1250it [00:01, 698.70it/s]
1250it [00:01, 735.21it/s]
1250it [00:02, 582.80it/s]
1250it [00:02, 601.57it/s]
1250it [00:01, 699.29it/s]
1250it [00:02, 603.48it/s]
1250it [00:01, 700.66it/s]
1250it [00:01, 726.45it/s]
1250it [00:01, 714.33it/s]
1

In [26]:
print(list_of_acc)

[49.52750000000004, 49.52750000000001, 49.52749999999996, 49.527499999999954, 49.527500000000025, 49.52750000000004, 49.52749999999997, 49.527499999999996, 49.52750000000006, 49.52750000000003, 49.52750000000001, 49.527499999999996, 49.527499999999996, 49.52749999999995, 49.52750000000004, 49.52749999999999, 49.52750000000002, 49.52750000000001, 49.527499999999996, 49.527499999999925, 49.52749999999997, 49.527499999999975, 49.5275, 49.52750000000004, 49.527499999999996, 49.52750000000003, 49.52750000000002, 49.527500000000074, 49.52749999999993, 49.527499999999996, 49.52749999999995, 49.527499999999975, 49.5275, 49.52750000000002, 49.527500000000025, 49.52749999999994, 49.52749999999996, 49.527499999999975, 49.527499999999996, 49.52749999999997, 49.527499999999975, 49.52749999999999, 49.527500000000025, 49.52749999999997, 49.52749999999999, 49.52749999999993, 49.52749999999995, 49.5275, 49.527499999999975, 49.52750000000004]


In [27]:
print(list_of_losses)

[0.4955268040060998, 0.4900342797994614, 0.4927634692788127, 0.49257207876443865, 0.49739585697650945, 0.49630328817367625, 0.49312547192573614, 0.4844312515139576, 0.4845646437048913, 0.48331157603263863, 0.48213051087856273, 0.48522799395322785, 0.48742447457313565, 0.4878999354958533, 0.48703286358118064, 0.48486076668500927, 0.4836996166229243, 0.48810735704898806, 0.48952973750829665, 0.4844412476062772, 0.4820624444007879, 0.48140893206596364, 0.48038919258117646, 0.482888132500648, 0.4858780835032456, 0.48968825011253336, 0.49014489669799827, 0.49000135726928695, 0.49187251852750785, 0.49287343341112094, 0.4901151842474946, 0.48796904115676953, 0.48550836976766637, 0.4855435858011246, 0.4886724588036531, 0.4883618970990176, 0.48862697075605405, 0.4870296754240994, 0.4928545437932014, 0.49479701517820374, 0.49396549699306513, 0.4941872516155243, 0.4950650214910509, 0.489362332355976, 0.4880467644810679, 0.48932889599800117, 0.4862655196905136, 0.488326912605762, 0.492863240599631

## Testing

In [28]:
from sklearn.metrics import accuracy_score
accuracy = []
for bidx, batch in tqdm(enumerate(test_loader)):
    x_test, y_test = batch[0], batch[1]
    x_test = x_test.float()
    y_test = y_test.float()
    output = net.forward(x_test)
    acc = compute_accuracy(output, y_test)
    accuracy.append(acc)

313it [00:00, 1366.16it/s]


In [29]:
def Average(lst): 
    return sum(lst) / len(lst) 
  
average = Average(accuracy) 

In [30]:
average

49.530750798722046